In [29]:
import os.path
from decimal import Decimal
from rdkit import Chem
from rdkit import RDLogger
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn
import pickle
import os
import sklearn
RDLogger.DisableLog('rdApp.*')
from sklearn.cluster import KMeans

## SET CONSTANTS

In [20]:
DATA_FOLDER = '/home/artem/dataset_gdb17'
SAMPLE_SIZE = 1000  #<-- data set size
MOL_SIZE_OF_INTEREST = 17  # number of atoms
#DATA_FILE_NAME = 'GDB17.50000000.smi'
DATA_FILE_NAME = str(MOL_SIZE_OF_INTEREST) + '.smi'
DATA_PATH =  DATA_FOLDER + '/' + str(SAMPLE_SIZE) + '/' + DATA_FILE_NAME

In [21]:
#example of k means for 1000 mols of the molecules with a size of 17

## DATA

In [22]:
DATA_PATH

'/home/artem/dataset_gdb17/1000/17.smi'

## LOAD DATA


In [23]:
suppl = Chem.SmilesMolSupplier(DATA_PATH)
smiles_list = [Chem.MolToSmiles(m) for m in suppl]
smiles_list[0:3]
# smiles_list is our raw data. now we need to convert them into some representation

['CCCC(C)OC(C=O)C(=O)C(C)(C)CCO',
 'CCc1cc(COCCC(N)=O)ncnc1=O',
 'CC(O)C1Oc2c(N)c(N)c(N)c(C#N)c2O1']

## GET FEATURES FOR EVERY MOLECULE


In [24]:
features = [Chem.RDKFingerprint(m) for m in suppl]

In [25]:
fp_array = np.array([[int(i) for i in Chem.RDKFingerprint(m).ToBitString()] for m in suppl], dtype=int)

In [26]:
fp_array


array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 1],
       ...,
       [0, 0, 1, ..., 0, 0, 1],
       [1, 1, 1, ..., 0, 1, 1],
       [0, 0, 1, ..., 0, 1, 1]])

In [31]:
kmeans = KMeans(n_clusters=100).fit(X=fp_array)

In [38]:
kmeans.labels_

kmeans.cluster_centers_

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 6.00000000e-01,  0.00000000e+00, -5.55111512e-17, ...,
         0.00000000e+00,  0.00000000e+00, -1.11022302e-16],
       [ 3.33333333e-01,  9.52380952e-02,  3.80952381e-01, ...,
         4.76190476e-02,  2.85714286e-01,  1.00000000e+00],
       ...,
       [ 2.85714286e-01,  1.42857143e-01,  5.00000000e-01, ...,
         0.00000000e+00,  2.85714286e-01,  1.00000000e+00],
       [ 0.00000000e+00,  5.00000000e-01,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [35]:
for cluster_center in kmeans.cluster_centers_:


# the_closest_cluster = kmeans.predict(X=fp_array)

In [37]:
# centers_and_index = kmeans.fit_predict(X=fp_array)